<a href="https://colab.research.google.com/github/Pop115/ojvindtriche/blob/main/Ojvindtriche.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Ojvindtriche**

Trouve le mot du jour de [ojvindix](https://www.ojvindix.fr/)

#### Téléchargement du dictionnaire et des vecteurs de mots

In [ ]:
!wget https://github.com/lovasoa/cemantriche/releases/download/v0/frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin

In [ ]:
!wget https://github.com/lovasoa/cemantriche/releases/download/v0/lexique-grammalecte-fr-v7.0.txt

#### Chargement en mémoire du dictionnaire

In [ ]:
!pip install gensim==4.2

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin", binary=True, unicode_errors="ignore")

In [ ]:
dico = pd.read_csv("lexique-grammalecte-fr-v7.0.txt", skiprows=15, sep='\t')
dico.sort_values("Fréquence", inplace=True)
dico = dico[(dico.Fréquence > 1e-5) & (dico.Étiquettes.str.match("nom|v[123]|adj")) & (dico.Lemme.str.len() > 2) & (dico.Flexion == dico.Lemme)]
dico = dico[dico.Lemme.isin(model.index_to_key)]
dico[["Lemme", "Fréquence"]]

#### Fonction d'évaluation de la proximité d'un mot au mot du jour

In [ ]:
import requests
import time
def req_word(word):
    r = requests.post("https://www.ojvindix.fr:3001/similarity", json={"value": word}).json()
    if "error" in r and "tapez trop vite" in r["error"]:
        time.sleep(.2)
        return req_word(word)
    return r.get("value", np.nan)

#### Recherche du mot secret

À chaque étape, on trie les mots du plus cohérent avec les distances obtenues jusqu'ici au moins cohérent,
et on essaie le plus cohérent.

In [ ]:
import time
t = time.time()
vocab = pd.Series(dico.Fréquence.values, index=dico.Lemme)
i = 0
while not vocab.empty:
    i += 1
    word = vocab.idxmax()
    print(f"{i:3d}. J'essaie “{word}”. ", end="")
    score = req_word(word)
    vocab.drop(index=word, inplace=True)
    if np.isnan(score):
        print(" Cémantix ne connaît pas ce mot.")
        continue
    if score == 1:
        print(f" GAGNÉ en {time.time() - t:.1f} secondes")
        break
    print(f"Score: {100*score:.2f}.")
    sims = vocab.index.map({
        word: sim
        for word, sim
        in model.most_similar(word, topn=len(model))
    })
    vocab /= np.abs(score-sims)